# Program Overview

The purpose of this notebook is to acquire the raw data necessary to analyze monthly article traffic from English Wikipedia articles published 07-01-2015 through 09-30-2023.

The source data comes from English Wikipedia, the text of which is licensed under "Creative Commons Attribution Share-Alike license" (https://en.wikipedia.org/wiki/Wikipedia:Text_of_the_Creative_Commons_Attribution-ShareAlike_3.0_Unported_License)

We also use the Wikimedia Foundation REST API which has the following
terms of use:
https://www.mediawiki.org/wiki/REST_API#Terms_and_conditions

The Pageviews API documentation can be found at the following link:
https://wikitech.wikimedia.org/wiki/Analytics/AQS/Pageviews

We will leverage code developed by Dr. David W. McDonald for use in Data 512  which is provided under Creative Commons CC-BY license. (https://creativecommons.org/ and https://creativecommons.org/licenses/by/4.0/)

Additionally, we will be using a list of Academy Award winning article titles provided by Dr. McDonald. A link to the list can be found here: https://drive.google.com/drive/folders/1lPJF73GX5Vyu2uAvT5VpAY-xGwP2fCCx

## API Pull Setup

The following sections of code will create a method to call the Pageviews API. The code was taken from Dr. McDonald's scripts noted in the Program Overview section above. Limited changes were made to lower-case variable names, update the range of dates for information pulls, and to limit line length where possible.

We begin by importing the necessary Python libraries. Users may not have "requests" installed, and can use pip to add it to their machine.

In [1]:
#Import python libraries
import json
import time
import urllib.parse
import requests
import pandas as pd

The following code will create constants leveraged in the API calls in addition to a list of test article titles to verify the code is working correctly.

In [2]:
#Creating list of constants leveraged in API calls

# The REST API 'pageviews' URL - this is the common URL/endpoint for all 'pageviews' API requests
api_request_pageviews_endpoint = 'https://wikimedia.org/api/rest_v1/metrics/pageviews/'


# This is a parameterized string that specifies what kind of pageviews request we are going to make
# In this case it will be a 'per-article' based request. The string is a format string so that we can
# replace each parameter with an appropriate value before making the request
api_request_per_article_params = 'per-article/{project}/{access}/{agent}/{article}/{granularity}/{start}/{end}'

# The Pageviews API asks that we not exceed 100 requests per second, we add a small delay to each request
api_latency_assumed = 0.002       # Assuming roughly 2ms latency on the API and network
api_throttle_wait = (1.0/100.0)-api_latency_assumed

# When making a request to the Wikimedia API they ask that you include your email address which will allow them
# to contact you if something happens - such as - your code exceeding rate limits - or some other error 
request_headers = {
    'User-Agent': 'ekrolen@uw.edu, University of Washington, MSDS DATA 512 - AUTUMN 2023',
}

# This is just a list of English Wikipedia article titles that we can use for example requests
ex_article_titles = [ 'Bison', 'Northern flicker', 'Red squirrel', 'Chinook salmon', 'Horseshoe bat' ]

# This template is used to map parameter values into the API_REQUST_PER_ARTICLE_PARAMS portion of an API request. 
# The dictionary has a field/key for each of the required parameters. In the example, below, we only vary the article name,
# so the majority of the fields can stay constant for each request. Of course, these values *could* be changed if necessary.
article_pageviews_params_template = {
    "project":     "en.wikipedia.org",
    "access":      "",      # this should be changed for the different access types
    "agent":       "user",
    "article":     "",             # this value will be set/changed before each request
    "granularity": "monthly",
    "start":       "2015070100",   # July 1, 2015 in YYYYMMDDHH format
    "end":         "2023093000"    # September 30, 2023 in YYYYMMDDHH format
}

As described in Dr. McDonald's code, "The API request will be made using one procedure. The idea is to make this reusable. The procedure is parameterized, but relies on the constants above for the important parameters. The underlying assumption is that this will be used to request data for a set of article pages. Therefore the parameter most likely to change is the article_title."

In [3]:
#Building the api request 

def request_pageviews_per_article(article_title = None, 
                                  access_method = None,
                                  endpoint_url = api_request_pageviews_endpoint, 
                                  endpoint_params = api_request_per_article_params,
                                  request_template = article_pageviews_params_template,
                                  headers = request_headers):

    # article title can be as a parameter to the call or in the request_template
    if article_title:
        request_template['article'] = article_title

    if not request_template['article']:
        raise Exception("Must supply an article title to make a pageviews request.")

    # Titles are supposed to have spaces replaced with "_" and be URL encoded
    article_title_encoded = urllib.parse.quote(request_template['article'].replace(' ','_'))
    request_template['article'] = article_title_encoded
    
    #Access method can also be a parameter in the call or in the request_template
    if access_method:
        request_template['access'] = access_method
    
    if not request_template['access']:
        raise Exception("Must supply an access method ('desktop', 'mobile-app', or 'mobile-web') to make a pageviews request.")
    
    # now, create a request URL by combining the endpoint_url with the parameters for the request
    request_url = endpoint_url+endpoint_params.format(**request_template)
    
    # make the request
    try:
        # we'll wait first, to make sure we don't exceed the limit in the situation where an exception
        # occurs during the request processing - throttling is always a good practice with a free
        # data source like Wikipedia - or other community sources
        if api_throttle_wait > 0.0:
            time.sleep(api_throttle_wait)
        response = requests.get(request_url, headers=headers)
        json_response = response.json()
    except Exception as e:
        print(e)
        json_response = None
    return json_response

The following code can be uncommented to verify the API call is working correctly.

In [4]:
'''print("Getting pageview data for: ",ex_article_titles[1])
views = request_pageviews_per_article(ex_article_titles[1], "mobile-app")

#print(json.dumps(views,indent=4))
print("Have %d months of pageview data"%(len(views['items'])))
for month in views['items']:
    print(json.dumps(month,indent=4))'''

'print("Getting pageview data for: ",ex_article_titles[1])\nviews = request_pageviews_per_article(ex_article_titles[1], "mobile-app")\n\n#print(json.dumps(views,indent=4))\nprint("Have %d months of pageview data"%(len(views[\'items\'])))\nfor month in views[\'items\']:\n    print(json.dumps(month,indent=4))'

The following section of code will build the list of articles we want to retrieve data for. See the Program Overview section for more information on the source of this list. 

thank_the_academy.AUG.2023.csv should be downloaded from https://docs.google.com/spreadsheets/d/1A1h_7KAo7KXaVxdScJmIVPTvjb3IuY9oZhNV4ZHxrxw/edit#gid=1229854301 and saved to this repository's "raw_data" directory before running this code.

In [5]:
#Create a dataframe from the csv
article_title_list = pd.read_csv('../raw_data/thank_the_academy.AUG.2023.csv')

#Save the names as a list
article_titles = list(article_title_list['name'])

## Acquire monthly mobile app access data

The following section of code will acquire monthly mobile app access data using the request_pageviews_per_article method and list of article titles built previously. The outputs are stored in a dictionary where the key is the article name (movie title) and the values are the timeseries data as returned by the API.

In [6]:
#Pulling the monthly mobile-app views for each article in the list
mobile_app_dict = {}
missing_titles = []
for i in range(len(article_titles)):
    if '/' in article_titles[i]:
        missing_titles.append(article_titles[i])
    else:
        views = request_pageviews_per_article(article_titles[i], 'mobile-app')
        mobile_app_dict[article_titles[i]] = views['items']
    if i%10 == 0:
        print(str(i)+" of "+str(len(article_titles))+" have been written")

0 of 1359 have been written
10 of 1359 have been written
20 of 1359 have been written
30 of 1359 have been written
40 of 1359 have been written
50 of 1359 have been written
60 of 1359 have been written
70 of 1359 have been written
80 of 1359 have been written
90 of 1359 have been written
100 of 1359 have been written
110 of 1359 have been written
120 of 1359 have been written
130 of 1359 have been written
140 of 1359 have been written
150 of 1359 have been written
160 of 1359 have been written
170 of 1359 have been written
180 of 1359 have been written
190 of 1359 have been written
200 of 1359 have been written
210 of 1359 have been written
220 of 1359 have been written
230 of 1359 have been written
240 of 1359 have been written
250 of 1359 have been written
260 of 1359 have been written
270 of 1359 have been written
280 of 1359 have been written
290 of 1359 have been written
300 of 1359 have been written
310 of 1359 have been written
320 of 1359 have been written
330 of 1359 have been

Next we will print out any article titles which could not be parsed by the Pageviews API due to a "/" present in their name. These articles will be omitted from our analysis as their information cannot be pulled using the API.

In [7]:
print("The following titles were unable to be parsed by the Pageviews API and will not be included in future analysis:")
print(missing_titles)

The following titles were unable to be parsed by the Pageviews API and will not be included in future analysis:
['Victor/Victoria']


We will now save the mobile-app pageview information in a JSON for future processing.

In [8]:
#Making mobile_app_json a JSON
mobile_app_json_object = json.dumps(mobile_app_dict, indent = 4) 

#Writing to the file
with open('../raw_data/mobile_app_pageviews.json', 'w') as outfile:
    outfile.write(mobile_app_json_object)

The following code verifies that the user got the full number of expected articles from the pageviews app, minus the ones which could not be pulled due to a "/" in the title.

In [11]:
#Testing if the # of keys in the mobile_app_json is equal to the number of article titles minus the titles which
#could not be pulled
if len(mobile_app_dict.keys()) == len(article_titles) - len(missing_titles):
    print("Correct number of articles captured in pageviews.")
else:
    print("ERROR - wrong number of articles captured in pageviews. Expecting "+str(len(article_titles) - len(missing_titles))+
         " and got: "+str(len(mobile_app_dict.keys()))) 

Correct number of articles captured in pageviews.


## Acquire monthly mobile web access data

The following section of code will acquire monthly mobile web access data using the request_pageviews_per_article method and list of article titles built previously. The outputs are stored in a dictionary where the key is the article name (movie title) and the values are the timeseries data as returned by the API.

In [12]:
#Pulling the monthly mobile-web views for each article in the list
mobile_web_dict = {}
missing_mobile_web_titles = []
for i in range(len(article_titles)):
    if '/' in article_titles[i]:
        missing_mobile_web_titles.append(article_titles[i])
    else:
        views = request_pageviews_per_article(article_titles[i], 'mobile-web')
        mobile_web_dict[article_titles[i]] = views['items']
    if i%10 == 0:
        print(str(i)+" of "+str(len(article_titles))+" have been written")

0 of 1359 have been written
10 of 1359 have been written
20 of 1359 have been written
30 of 1359 have been written
40 of 1359 have been written
50 of 1359 have been written
60 of 1359 have been written
70 of 1359 have been written
80 of 1359 have been written
90 of 1359 have been written
100 of 1359 have been written
110 of 1359 have been written
120 of 1359 have been written
130 of 1359 have been written
140 of 1359 have been written
150 of 1359 have been written
160 of 1359 have been written
170 of 1359 have been written
180 of 1359 have been written
190 of 1359 have been written
200 of 1359 have been written
210 of 1359 have been written
220 of 1359 have been written
230 of 1359 have been written
240 of 1359 have been written
250 of 1359 have been written
260 of 1359 have been written
270 of 1359 have been written
280 of 1359 have been written
290 of 1359 have been written
300 of 1359 have been written
310 of 1359 have been written
320 of 1359 have been written
330 of 1359 have been

Next we will print out any article titles which could not be parsed by the Pageviews API due to a "/" present in their name. These articles will be omitted from our analysis as their information cannot be pulled using the API.

In [13]:
print("The following titles were unable to be parsed by the Pageviews API and will not be included in future analysis:")
print(missing_mobile_web_titles)

The following titles were unable to be parsed by the Pageviews API and will not be included in future analysis:
['Victor/Victoria']


We will now save the mobile-web pageview information in a JSON for future processing.

In [14]:
#Making mobile_web_json a JSON
mobile_web_json_object = json.dumps(mobile_web_dict, indent = 4) 

#Writing to the file
with open('../raw_data/mobile_web_pageviews.json', 'w') as outfile:
    outfile.write(mobile_web_json_object)

The following code verifies that the user got the full number of expected articles from the pageviews app, minus the ones which could not be pulled due to a "/" in the title.

In [15]:
#Testing if the # of keys in the mobile_web_dict is equal to the number of article titles minus the titles which
#could not be pulled
if len(mobile_web_dict.keys()) == len(article_titles) - len(missing_mobile_web_titles):
    print("Correct number of articles captured in pageviews.")
else:
    print("ERROR - wrong number of articles captured in pageviews. Expecting "+
          str(len(article_titles) - len(missing_mobile_web_titles))+" and got: "+str(len(mobile_web_dict.keys()))) 

Correct number of articles captured in pageviews.


## Acquire desktop access data

The following section of code will acquire monthly desktop access data using the request_pageviews_per_article method and list of article titles built previously. The outputs are stored in a dictionary where the key is the article name (movie title) and the values are the timeseries data as returned by the API.

In [16]:
#Pulling the monthly desktop views for each article in the list
desktop_dict = {}
missing_desktop_titles = []
for i in range(len(article_titles)):
    if '/' in article_titles[i]:
        missing_desktop_titles.append(article_titles[i])
    else:
        views = request_pageviews_per_article(article_titles[i], 'desktop')
        desktop_dict[article_titles[i]] = views['items']
    if i%10 == 0:
        print(str(i)+" of "+str(len(article_titles))+" have been written")

0 of 1359 have been written
10 of 1359 have been written
20 of 1359 have been written
30 of 1359 have been written
40 of 1359 have been written
50 of 1359 have been written
60 of 1359 have been written
70 of 1359 have been written
80 of 1359 have been written
90 of 1359 have been written
100 of 1359 have been written
110 of 1359 have been written
120 of 1359 have been written
130 of 1359 have been written
140 of 1359 have been written
150 of 1359 have been written
160 of 1359 have been written
170 of 1359 have been written
180 of 1359 have been written
190 of 1359 have been written
200 of 1359 have been written
210 of 1359 have been written
220 of 1359 have been written
230 of 1359 have been written
240 of 1359 have been written
250 of 1359 have been written
260 of 1359 have been written
270 of 1359 have been written
280 of 1359 have been written
290 of 1359 have been written
300 of 1359 have been written
310 of 1359 have been written
320 of 1359 have been written
330 of 1359 have been

Next we will print out any article titles which could not be parsed by the Pageviews API due to a "/" present in their name. These articles will be omitted from our analysis as their information cannot be pulled using the API.

In [17]:
print("The following titles were unable to be parsed by the Pageviews API and will not be included in future analysis:")
print(missing_desktop_titles)

The following titles were unable to be parsed by the Pageviews API and will not be included in future analysis:
['Victor/Victoria']


We will now save the desktop pageview information in a JSON for future processing.

In [18]:
#Making desktop_json a JSON
desktop_json_object = json.dumps(desktop_dict, indent = 4) 

#Writing to the file
with open('../raw_data/desktop_pageviews.json', 'w') as outfile:
    outfile.write(desktop_json_object)

The following code verifies that the user got the full number of expected articles from the pageviews app, minus the ones which could not be pulled due to a "/" in the title.

In [19]:
#Testing if the # of keys in the desktop_dict is equal to the number of article titles minus the titles which
#could not be pulled
if len(desktop_dict.keys()) == len(article_titles) - len(missing_desktop_titles):
    print("Correct number of articles captured in pageviews.")
else:
    print("ERROR - wrong number of articles captured in pageviews. Expecting "+
          str(len(article_titles) - len(missing_desktop_titles))+" and got: "+str(len(desktop_dict.keys()))) 

Correct number of articles captured in pageviews.
